# Stairs Circuit and its gradients evaluated using native simulators

A "quantum cost function" is the mean value of a
Hermitian operator, wherein that mean value is calculated empirically from
the data yielded by a physical quantum computer. (In this notebook, however, we fake that data
with a Qubiter simulator. Just warming up. Brrmm! Brrmm! Everything is setup
so that calls to a real physical qc such as Rigetti's qc can be easily substituted for the 
calls to the Qubiter simulator)

A Stairs circuit is a quantum circuit that in its full generality can 
parametrize a completely general quantum state vector. 

In this notebook, we consider a special quantum cost function in which
a Stairs circuit provides the state vector with which the mean values are calculated.
I like to say that the Stairs circuit is the kernel of the quantum cost function. Our ultimate intention is to
minimize this cost function using gradient descent. But to do that, we must first calculate
the derivatives (gradients) of the cost function. That is what we will do in this notebook: calculate all the derivatives of a quantum cost function whose kernel is a Stairs circuit.

 In particular,
this notebook runs through their paces 4 Qubiter classes that increasingly build on each other

*  `StairsCkt_writer`, writes English and Picture files for a Stairs quantum circuit
*  `StairsDeriv_writer`, writes English and Picture files for several quantum circuits that are needed to evaluate the derivatives of a Stairs circuit
*  `StairsDeriv_native`, evaluates the 4 derivatives of a single gate of a Stairs circuit. It uses native Qubiter simulators to do this. Qubiter also has an analogous class `StairsDeriv_rigetti` that uses Rigetti simulators or their real physical qc. We will demo that class in a future notebook.
*  `StairsAllDeriv_native`, evaluates all the derivatives, for all the gates of a Stairs circuit. It uses native Qubiter simulators to do this. Qubiter also has an analogous class `StairsAllDeriv_rigetti` that uses Rigetti simulators or their real physical qc.  We will demo that class in a future notebook.

For an explanation of the theory behind the software that is being demo-ed in this notebook, see the following
pdf included with the Qubiter distribution.

>Title:
Calculation of the Gradient of a Quantum Cost Function using "Threading".
Application of these "threaded gradients" to a
Quantum Neural Net
inspired by
Quantum Bayesian Networks, 

> https://github.com/artiste-qb-net/qubiter/blob/master/adv_applications/threaded_grad.pdf

First change your working directory to the Qubiter directory in your computer, and add its path to the path environment variable.

In [1]:
import os
import sys
print(os.getcwd())
os.chdir('../../')
print(os.getcwd())
sys.path.insert(0,os.getcwd())

C:\Users\rrtuc\Desktop\backedup\python-projects\qubiter\qubiter\jupyter_notebooks
C:\Users\rrtuc\Desktop\backedup\python-projects\qubiter


# class StairsCkt_writer

In [2]:
from qubiter.adv_applications.StairsCkt_writer import *

loaded OneBitGates, WITHOUT autograd.numpy


In [3]:
# print docstring of the class
print(StairsCkt_writer.__doc__)


    This class is a subclass of class SEO_writer and it writes a "Stairs
    Circuit". For example, this is what the Picture file of a Stairs Circuit 
    looks like for num_bits = 3 
    
    U   |   |
    O---U   |  
    @---U   |   
    O---O---U   
    O---@---U   
    @---O---U  
    @---@---U 
       
    Here, U is a general U(2) matrix with 4 parameters, all of which can be 
    made into placeholder variables. If each U is represented by a node and 
    the controls of each U represent its parents, then this quantum circuit 
    can be represented by a fully connected Quantum Bayesian Network (QB 
    net). (See my >10 year old blog called "Quantum Bayesian Networks" for 
    more info than you would ever want to know about QB nets). 
    
    This class can also be asked to construct a QB net that is **not** fully 
    connected, by limiting the number of controls for a given U to fewer 
    than all the ones to its left. For example, suppose that in the 
    num_bits=3 case

Before using class `StairsCkt_writer` to write a quantum circuit, 
we introduce some static methods belonging to this class
that are very helpful, not just with 
this class, but with all the other classes related to the stairs circuit.

`get_gate_str_to_rads_list()` returns a dictionary that maps a gate string to
a list of 4 radians. The gate string is the unique name we will give to each gate of 
the Stairs circuit, and the 4 radians are the values of the angles
characterizing a general U(2) transformation, $(t_0, t_1, t_2, t_3)$
in $e^{i(t_0, + t_1\sigma_X + t_2\sigma_Y+ t_3\sigma_Z)}$,
where $\sigma_X, \sigma_Y, \sigma_Z$ are the Pauli matrices.
The user can select among 3 fill types, either 'const' for a constant value,
'rand' for random values, or '#int' for a hash followed by a unique int. 

In [4]:
num_bits = 3
for fill_type in ['const', 'rand', '#int']:
    di = StairsCkt_writer.get_gate_str_to_rads_list(
        num_bits, fill_type, rads_const=.3)
    pp.pprint(di)

OrderedDict([('prior', [0.3, 0.3, 0.3, 0.3]),
             ('2F', [0.3, 0.3, 0.3, 0.3]),
             ('2T', [0.3, 0.3, 0.3, 0.3]),
             ('2F1F', [0.3, 0.3, 0.3, 0.3]),
             ('2F1T', [0.3, 0.3, 0.3, 0.3]),
             ('2T1F', [0.3, 0.3, 0.3, 0.3]),
             ('2T1T', [0.3, 0.3, 0.3, 0.3])])
OrderedDict([('prior',
              [0.8125220078555554,
               0.902382106984018,
               0.9538707565997522,
               0.9259035602326606]),
             ('2F',
              [1.875639771864806,
               3.875554421094597,
               0.23481147823757959,
               3.4864875211667887]),
             ('2T',
              [5.3448141278808405,
               0.8841575449597163,
               4.269585543076376,
               0.8254633971566671]),
             ('2F1F',
              [2.91415733851564,
               3.1171682669128384,
               4.534609791939031,
               0.15229543929944556]),
             ('2F1T',
              [1.

Above, we asked `get_gate_str_to_rads_list()`
to print a dictionary  for a fully connected Quantum Bayesian network,
one in which every gate has all qubits to the left of its U as parents. It is 
also possible to ask `get_gate_str_to_rads_list()`
to print a dictionary for a non-fully connected QB net,
wherein only some, not all, of the qubits to the left of the U are parents.
This can be done by specifying a value for 

`u2_bit_to_higher_bits`

For a 3 qubit case, if one sets that variable to

`{0: [1, 2], 1: [2], 2: []}` 

or to None, one gets a fully connected QB net.
This is what happens if one sets it to 

`{0: [2], 1: [2], 2: []}`

instead:

In [5]:
u2_bit_to_higher_bits = {0: [2], 1: [2], 2: []}
di = StairsCkt_writer.get_gate_str_to_rads_list(
        num_bits, "#int", u2_bit_to_higher_bits=u2_bit_to_higher_bits)
pp.pprint(di)

OrderedDict([('prior', ['#50', '#51', '#52', '#53']),
             ('2F', ['#500', '#501', '#502', '#503']),
             ('2T', ['#510', '#511', '#512', '#513']),
             ('2F1_', ['#5050', '#5051', '#5052', '#5053']),
             ('2T1_', ['#5150', '#5151', '#5152', '#5153'])])


Another helpful static method of the class `StairsCkt_writer`  is `get_var_num_to_rads()`.
If the input `di` equals a `gate_str_to_rads_list` dictionary,
then this method extracts all the ints to the right of a hash
and maps them to a float. The float can be either a constant (fill_type='const')
or a random number (fill_type='rand')

In [6]:
vn_to_r = StairsCkt_writer.get_var_num_to_rads(di,
                                               fill_type='const',
                                               rads_const=.3)
pp.pprint(vn_to_r)

{50: 0.3,
 51: 0.3,
 52: 0.3,
 53: 0.3,
 500: 0.3,
 501: 0.3,
 502: 0.3,
 503: 0.3,
 510: 0.3,
 511: 0.3,
 512: 0.3,
 513: 0.3,
 5050: 0.3,
 5051: 0.3,
 5052: 0.3,
 5053: 0.3,
 5150: 0.3,
 5151: 0.3,
 5152: 0.3,
 5153: 0.3}


Another helpful static method of the class `StairsCkt_writer`  is `make_array_from_gate_str_to_rads_list()`.
If the input `di` equals a `gate_str_to_rads_list` dictionary,
then this method converts that dictionary into a numpy array. (It works even for strings!)

In [7]:
arr = StairsCkt_writer.make_array_from_gate_str_to_rads_list(di)
print("arr=\n", arr)

arr=
 [['#50' '#51' '#52' '#53']
 ['#500' '#501' '#502' '#503']
 ['#510' '#511' '#512' '#513']
 ['#5050' '#5051' '#5052' '#5053']
 ['#5150' '#5151' '#5152' '#5153']]


After much ado, we finally call the constructor of class  `StairsCkt_writer`.
This writes English and Picture files in the `io_folder`.

In [8]:
num_bits = 4
gate_str_to_rads_list = StairsCkt_writer.get_gate_str_to_rads_list(
    num_bits, '#int')
file_prefix = 'qubiter/io_folder/stairs_writer_test'
emb = CktEmbedder(num_bits, num_bits)
wr = StairsCkt_writer(gate_str_to_rads_list, file_prefix, emb)
wr.close_files()

Next, we ask the writer `wr` to print the English and Picture files that it just created

In [9]:
wr.print_eng_file()
wr.print_pic_file()

U_2_	#50	#51	#52	#53	AT	3
U_2_	#500	#501	#502	#503	AT	2	IF	3F
U_2_	#510	#511	#512	#513	AT	2	IF	3T
U_2_	#5000	#5001	#5002	#5003	AT	1	IF	3F	2F
U_2_	#5010	#5011	#5012	#5013	AT	1	IF	3F	2T
U_2_	#5100	#5101	#5102	#5103	AT	1	IF	3T	2F
U_2_	#5110	#5111	#5112	#5113	AT	1	IF	3T	2T
U_2_	#50000	#50001	#50002	#50003	AT	0	IF	3F	2F	1F
U_2_	#50010	#50011	#50012	#50013	AT	0	IF	3F	2F	1T
U_2_	#50100	#50101	#50102	#50103	AT	0	IF	3F	2T	1F
U_2_	#50110	#50111	#50112	#50113	AT	0	IF	3F	2T	1T
U_2_	#51000	#51001	#51002	#51003	AT	0	IF	3T	2F	1F
U_2_	#51010	#51011	#51012	#51013	AT	0	IF	3T	2F	1T
U_2_	#51100	#51101	#51102	#51103	AT	0	IF	3T	2T	1F
U_2_	#51110	#51111	#51112	#51113	AT	0	IF	3T	2T	1T

U   |   |   |   
O---U   |   |   
@---U   |   |   
O---O---U   |   
O---@---U   |   
@---O---U   |   
@---@---U   |   
O---O---O---U   
O---O---@---U   
O---@---O---U   
O---@---@---U   
@---O---O---U   
@---O---@---U   
@---@---O---U   
@---@---@---U   



# class StairsDerivCkt_writer 

In [10]:
from qubiter.adv_applications.StairsDerivCkt_writer import *

In [11]:
# print docstring of the class
print(StairsDerivCkt_writer.__doc__)


    This class is a subclass of `SEO_writer`. It writes several intermediary
    stairs derivative circuits that will be used in class
    `StairsDeriv_native` for calculating the gradients of a quantum cost
    function (mean hamiltonian).

    Suppose U = exp[i*(t_0 + t_1*sigx + t_2*sigy + t_3*sigz)], where sigx,
    sigy, sigz are the Pauli matrices and t_r for r in range(4) are 4 real
    parameters. To take the derivative wrt t_r of a given multi-controlled
    gate U in a stairs circuit, we need to evaluate several circuits (we
    call them dparts, which stands for derivative parts). Say, for instance,
    that GATE= @---O---+---U. To calculate d/dt_r GATE(t_0, t_1, t_2, t_3),
    for r=0,1, 2, 3, we need to calculate a new circuit wherein the GATE in
    the parent circuit is replaced by

    sum_k  c_k  @---@---O---+---U_k

    (which is said to have `has_neg_polarity`=False) and

    sum_k  c_k  @---@---O---+---U_k
                Z---@---O   |   |

    (which is said to hav

In [12]:
num_bits = 4
parent_num_bits = num_bits - 1  # one bit for ancilla
gate_str_to_rads_list = StairsCkt_writer.\
get_gate_str_to_rads_list(
    parent_num_bits, 'const', rads_const=np.pi/2)
file_prefix = 'qubiter/io_folder/stairs_deriv_writer_test'
emb = CktEmbedder(num_bits, num_bits)

One of the inputs to the constructor of class `StairsDerivCkt_writer`
is `deriv_gate_str`, which should be a well-formed gate string specifying
what gate of the stairs circuit we want to differentiate.
For that, we will use the second key of the `gate_str_to_rads_list` dictionary

In [13]:
deriv_gate_str = list(gate_str_to_rads_list.keys())[2]
print(deriv_gate_str)

2T


We next call the constructor of class
`StairsDerivCkt_writer` for two typical cases, 
and then print the English and Picutre files for each of the 2 cases

In [14]:
for deriv_direc, dpart_name, has_neg_polarity in \
        [(0, 'single', None), (3, 's', True)]:
    wr = StairsDerivCkt_writer(deriv_gate_str,
                               has_neg_polarity,
                               deriv_direc,
                               dpart_name,
                               gate_str_to_rads_list,
                               file_prefix, emb)
    wr.close_files()
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%")
    wr.print_eng_file()
    wr.print_pic_file()

%%%%%%%%%%%%%%%%%%%%%%%%%%
U_2_	90.0	90.0	90.0	90.0	AT	2
U_2_	90.0	90.0	90.0	90.0	AT	1	IF	2F
HAD2	AT	3
U_2_	180.0	90.0	90.0	90.0	AT	1	IF	3T	2T
U_2_	90.0	90.0	90.0	90.0	AT	0	IF	2F	1F
U_2_	90.0	90.0	90.0	90.0	AT	0	IF	2F	1T
U_2_	90.0	90.0	90.0	90.0	AT	0	IF	2T	1F
U_2_	90.0	90.0	90.0	90.0	AT	0	IF	2T	1T

|   U   |   |   
|   O---U   |   
H   |   |   |   
@---@---U   |   
|   O---O---U   
|   O---@---U   
|   @---O---U   
|   @---@---U   

%%%%%%%%%%%%%%%%%%%%%%%%%%
U_2_	90.0	90.0	90.0	90.0	AT	2
U_2_	90.0	90.0	90.0	90.0	AT	1	IF	2F
HAD2	AT	3
U_2_	90.0	0.0	0.0	90.0	AT	1	IF	3T	2T
SIGZ	AT	3	IF	2T
U_2_	90.0	90.0	90.0	90.0	AT	0	IF	2F	1F
U_2_	90.0	90.0	90.0	90.0	AT	0	IF	2F	1T
U_2_	90.0	90.0	90.0	90.0	AT	0	IF	2T	1F
U_2_	90.0	90.0	90.0	90.0	AT	0	IF	2T	1T

|   U   |   |   
|   O---U   |   
H   |   |   |   
@---@---Uz  |   
Z---@   |   |   
|   O---O---U   
|   O---@---U   
|   @---O---U   
|   @---@---U   



# class StairsDeriv_native

In [15]:
from qubiter.adv_applications.StairsDeriv_native import *

In [16]:
# print docstring of the class
print(StairsDeriv_native.__doc__)


    This class is a child of StairsDeriv. Its main purpose is to override
    the method get_mean_val() of its abstract parent class StairsDeriv. In
    this class, the simulation necessary to evaluate the output of
    get_mean_val() is done by native, Qubiter simulators.

    Attributes
    ----------

    


In [17]:
num_bits = 4
parent_num_bits = num_bits - 1  # one bit for ancilla

# u2_bit_to_higher_bits = None
u2_bit_to_higher_bits = {0: [2], 1: [2], 2: []}
gate_str_to_rads_list = StairsCkt_writer.\
    get_gate_str_to_rads_list(parent_num_bits,
        '#int', rads_const=np.pi/2,
        u2_bit_to_higher_bits=u2_bit_to_higher_bits)
pp.pprint(gate_str_to_rads_list)

OrderedDict([('prior', ['#50', '#51', '#52', '#53']),
             ('2F', ['#500', '#501', '#502', '#503']),
             ('2T', ['#510', '#511', '#512', '#513']),
             ('2F1_', ['#5050', '#5051', '#5052', '#5053']),
             ('2T1_', ['#5150', '#5151', '#5152', '#5153'])])


In [18]:
deriv_gate_str = list(gate_str_to_rads_list.keys())[2]
print(deriv_gate_str)

2T


In [19]:
file_prefix = 'qubiter/io_folder/stairs_deriv_native_test'

The Hamiltonian `hamil` is entered as an object of class `QubitOperator`
of the open source library `OpenFermion`. The class 
constructor simplifies the input. Once simplified, `hamil` must be a 
linear combination with real coefficients of "pauli strings"

In [20]:
hamil = QubitOperator('X1 Y0 X1 Y1', .4) +\
    QubitOperator('Y2 X1', .7)
print(hamil)

0.4 [Y0 Y1] +
0.7 [X1 Y2]


Creating an object `der` of `StairsCkt_writer` doesn't do the whole trick. You
then need to call `der.get_mean_val(var_num_to_rads)` to get a list of the 4 partial derivatives wrt
the 4 parameters of the U(2) transformation for the gate called `deriv_gate_str`

In [21]:
der = StairsDeriv_native(deriv_gate_str,
                         gate_str_to_rads_list, file_prefix,
                         parent_num_bits, hamil)

var_num_to_rads = StairsCkt_writer.get_var_num_to_rads(
    gate_str_to_rads_list, 'const', rads_const=np.pi/2)

partials_list = der.get_mean_val(var_num_to_rads)
print('partials_list=', partials_list)


partials_list= [-0.21348209139938915, 0.1709965230969095, 0.21797739856238538, 0.20955227323123557]


# class StairsAllDeriv_native

In [22]:
from qubiter.adv_applications.StairsAllDeriv_native import *

In [23]:
# print docstring of the class
print(StairsAllDeriv_native.__doc__)


    This class is a child of StairsDeriv_native. For the parent class,
    the get_mean_val() method returns a list of 4 partial derivatives
    belonging to a particular gate string (a gate_str is a key in
    gate_str_to_rads_list). For this class, get_mean_val() returns an
    ordered dictionary mapping each gate_str to its 4 partials.

    Attributes
    ----------
    deriv_gate_str : str

    


In [24]:
num_bits = 4
parent_num_bits = num_bits - 1  # one bit for ancilla

# u2_bit_to_higher_bits = None
u2_bit_to_higher_bits = {0: [2], 1: [2], 2: []}
gate_str_to_rads_list = StairsCkt_writer.\
    get_gate_str_to_rads_list(parent_num_bits,
        '#int', rads_const=np.pi/2,
        u2_bit_to_higher_bits=u2_bit_to_higher_bits)
pp.pprint(gate_str_to_rads_list)

file_prefix = 'qubiter/io_folder/stairs_all_deriv_native_test'

hamil = QubitOperator('Y0 X1', .4) +\
    QubitOperator('X0', .7)

der = StairsAllDeriv_native(gate_str_to_rads_list, file_prefix,
                         parent_num_bits, hamil)

var_num_to_rads = StairsCkt_writer.get_var_num_to_rads(
    gate_str_to_rads_list, 'const', rads_const=np.pi/2)

gate_str_to_partials_list = der.get_mean_val(var_num_to_rads)
pp.pprint(gate_str_to_partials_list)


OrderedDict([('prior', ['#50', '#51', '#52', '#53']),
             ('2F', ['#500', '#501', '#502', '#503']),
             ('2T', ['#510', '#511', '#512', '#513']),
             ('2F1_', ['#5050', '#5051', '#5052', '#5053']),
             ('2T1_', ['#5150', '#5151', '#5152', '#5153'])])
OrderedDict([('prior',
              [0.0,
               0.10987285109716702,
               0.10987285109716702,
               0.06330213907425795]),
             ('2F',
              [-0.3154269436894809,
               0.19253599609389324,
               0.15844289259167696,
               0.05719217691065051]),
             ('2T',
              [-0.31077863786465393,
               0.02411052458829621,
               0.019841179494609645,
               0.007161951092978812]),
             ('2F1_',
              [-0.13581869634833232,
               0.3904738483600583,
               0.5194610247254097,
               0.5773185765431391]),
             ('2T1_',
              [-0.2882870043434722,
 